In [1]:
import numpy as np 
import os
import matplotlib.pyplot as plt
from imutils import paths
import io
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
dataset= r'C:\Users\infinity\Desktop\Mask Detection\data'
imagepaths=list(paths.list_images(dataset))

In [3]:
imagepaths

['C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\0-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\1-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\10-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\100-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\101-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\103-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\104-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\105-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\106-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\107-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\108-with-mask.jpg',
 'C:\\Users\\infinity\\Desktop\\Mask Detection\\data\\with mask\\109-w

In [4]:
data=[]
labels=[]

for i in imagepaths:
    label=i.split(os.path.sep)[-2]
    labels.append(label)
    image=load_img(i,target_size=(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)

In [5]:
labels

['with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with mask',
 'with

In [6]:
data= np.array(data, dtype='float32')
labels=np.array(labels)

In [7]:
data

array([[[[-0.5529412 , -0.5058824 , -0.7254902 ],
         [-0.6392157 , -0.5764706 , -0.7647059 ],
         [-0.654902  , -0.58431375, -0.7490196 ],
         ...,
         [-0.6784314 , -0.654902  , -0.8039216 ],
         [-0.75686276, -0.7254902 , -0.9372549 ],
         [-0.7176471 , -0.6784314 , -0.85882354]],

        [[-0.6       , -0.5529412 , -0.75686276],
         [-0.64705884, -0.58431375, -0.77254903],
         [-0.6392157 , -0.5686275 , -0.73333335],
         ...,
         [-0.49019605, -0.4588235 , -0.5764706 ],
         [-0.8039216 , -0.7647059 , -0.94509804],
         [-0.70980394, -0.64705884, -0.827451  ]],

        [[-0.62352943, -0.5764706 , -0.78039217],
         [-0.6392157 , -0.5764706 , -0.7647059 ],
         [-0.62352943, -0.5529412 , -0.7176471 ],
         ...,
         [-0.6       , -0.5529412 , -0.6627451 ],
         [-0.827451  , -0.7647059 , -0.94509804],
         [-0.7019608 , -0.6313726 , -0.78039217]],

        ...,

        [[-0.5294118 , -0.4352941 , -0

In [8]:
labels

array(['with mask', 'with mask', 'with mask', ..., 'without mask',
       'without mask', 'without mask'], dtype='<U12')

In [9]:
lb= LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

In [10]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [11]:
Xtrain, Xtest, Ytrain, Ytest= train_test_split(data, labels, test_size=0.2, random_state=10, stratify=labels)

In [12]:
print('shape of Xtrain:', Xtrain.shape)
print('shape of Ytrain:', Ytrain.shape)
print('shape of Xtest:', Xtest.shape)
print('shape of Ytest:', Ytest.shape)

shape of Xtrain: (1100, 224, 224, 3)
shape of Ytrain: (1100, 2)
shape of Xtest: (276, 224, 224, 3)
shape of Ytest: (276, 2)


In [13]:
aug= ImageDataGenerator(rotation_range=0.2, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, horizontal_flip=True, vertical_flip=True, fill_mode='nearest')

In [14]:
baseModel= MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))

In [15]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [16]:
headModel= baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='Flatten')(headModel)
headModel=Dense(128, activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)

model=Model(inputs=baseModel.input, outputs=headModel)

In [17]:
for layers in baseModel.layers:
    layers.trainable=False

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [21]:
learning_rate=0.001
Epochs=5
batch_size=32
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.fit(aug.flow(Xtrain,Ytrain, batch_size), steps_per_epoch=len(Xtrain)//batch_size, validation_data=(Xtest, Ytest),
          validation_steps=len(Xtest)//batch_size, epochs=Epochs)

model.save(r'C:\Users\infinity\Desktop\Mask Detection\model\maskdetection_model.model')

Epoch 1/5
34/34 [==============================] - 39s 1s/step - loss: 0.2692 - accuracy: 0.8942 - val_loss: 0.0588 - val_accuracy: 0.9819
Epoch 2/5
34/34 [==============================] - 34s 989ms/step - loss: 0.0640 - accuracy: 0.9803 - val_loss: 0.0494 - val_accuracy: 0.9819
Epoch 3/5
34/34 [==============================] - 34s 1s/step - loss: 0.0653 - accuracy: 0.9752 - val_loss: 0.0444 - val_accuracy: 0.9819
Epoch 4/5
34/34 [==============================] - 34s 1s/step - loss: 0.0397 - accuracy: 0.9860 - val_loss: 0.0374 - val_accuracy: 0.9891
Epoch 5/5
34/34 [==============================] - 34s 992ms/step - loss: 0.0379 - accuracy: 0.9878 - val_loss: 0.0323 - val_accuracy: 0.9891
INFO:tensorflow:Assets written to: C:\Users\infinity\Desktop\Mask Detection\model\maskdetection_model.model\assets


C:\Users\infinity\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\infinity\anaconda3\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [22]:
predict= model.predict(Xtest, batch_size)
predict=np.argmax(predict, axis=1)
print(classification_report(Ytest.argmax(axis=1), predict, target_names=lb.classes_))

              precision    recall  f1-score   support

   with mask       1.00      0.98      0.99       138
without mask       0.98      1.00      0.99       138

    accuracy                           0.99       276
   macro avg       0.99      0.99      0.99       276
weighted avg       0.99      0.99      0.99       276

